# RAG class

In [3]:
import openai
import pandas as pd
from langchain_openai import AzureChatOpenAI
import os
from configparser import ConfigParser
import shutil
c = ConfigParser()
c.read(r"C:\workspace\APIKEY_personal.ini")

['C:\\workspace\\APIKEY_personal.ini']

In [16]:
rag_config = {
    "api_key":c["AZURE_4o-mini"]["API_KEY"],
    "api_base":c["AZURE_4o-mini"]["OPENAI_API_BASE"],
    "api_version":c["AZURE_4o-mini"]["OPENAI_API_VERSION"],
    "deployment": c["AZURE_4o-mini"]["CHATGPT_MODEL"],
    "langsmith_key": c["KEY"]["LANGSMITH_PERSONAL"]
}

In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import AzureChatOpenAI,OpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
import shutil

class RAG:
    def __init__(self,config=None,chromadb_folder="./chroma_db") -> None:
        # save config settings to connect to Azure 4o-mini
        self.config = config
        # chromadb path
        self.chromadb_folder = chromadb_folder
        ###### some constants
        # chunksize
        self.chunksize = 1000
        # overlap
        self.overlap = 200
        # embedding model
        self.model_id = "BAAI/bge-small-en-v1.5"

        os.environ["OPENAI_API_KEY"] = self.config["api_key"]
        os.environ["OPENAI_API_VERSION"] = self.config["api_version"]
        os.environ["AZURE_OPENAI_ENDPOINT"] = self.config["api_base"]

        # create embedding function
        self.createEmbeddingFunction()

    @staticmethod
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    def add_website(self,webpath):
        # load website
        loader = WebBaseLoader(web_paths=(webpath,))
        loader.requests_kwargs = {"verify": False}
        docs = loader.load()
        # split the text into chuncks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=self.chunksize, chunk_overlap=self.overlap)
        splits = text_splitter.split_documents(docs)
        self.vectorstore = Chroma.from_documents(
            documents=splits, embedding=self.embedding_function)
        self.retriever = self.vectorstore.as_retriever()

    def add_docx(self,filepath):
        pass

    def createEmbeddingFunction(self):
        self.embedding_function = HuggingFaceEmbeddings(model_name=self.model_id, model_kwargs={'device': 'cpu'})


    def clearDatabase(self):
        # remove the existing RAG database
        shutil.rmtree(self.chromadb_folder)

    def setupLLM(self,local=False):
        self.prompt = hub.pull("rlm/rag-prompt",api_key=self.config["langsmith_key"])

        if local == True:
            self.llm = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")
        else:
            self.llm = AzureChatOpenAI(
                deployment_name=self.config["deployment"],
                model_name=self.config["deployment"], 
                temperature=0,
                max_tokens=None,
                timeout=None,
                max_retries=2)

        self.rag_chain = (
            {"context": self.retriever | self.format_docs, "question": RunnablePassthrough()}
            | self.prompt
            | self.llm
            | StrOutputParser())
    
    def invoke(self,question):
        print("invoking...")
        result = self.rag_chain.invoke(question)
        print(result)
        print("Complete")
        return result


In [18]:
r = RAG(config=rag_config)

c:\Users\TristramArmour\anaconda3\envs\learning\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
r.add_website("https://devlog.tublian.com/tublian-open-source-internship-cohort2-a-path-to-software-development-mastery")

c:\Users\TristramArmour\anaconda3\envs\learning\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'devlog.tublian.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [24]:
r.setupLLM(local=True)

In [25]:
a = r.invoke("How long is the Open Source internship?")

invoking...
The Open Source internship lasts for 30 days, starting on December 18th, 2023. Participants are expected to dedicate 6-8 hours a week to complete the tasks.
Complete
